In [ ]:
%%writefile cuda_bitonic_final.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <chrono>
#include <climits>

using namespace std;


__global__ void bitonicKernel(int* arr, int j, int k, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int pair = idx ^ j;

    if (pair > idx && idx < n && pair < n) {
        bool ascending = ((idx & k) == 0);
        if (ascending) {
            if (arr[idx] > arr[pair]) {
                int tmp = arr[idx];
                arr[idx] = arr[pair];
                arr[pair] = tmp;
            }
        } else {
            if (arr[idx] < arr[pair]) {
                int tmp = arr[idx];
                arr[idx] = arr[pair];
                arr[pair] = tmp;
            }
        }
    }
}


vector<int> generateRandomArray(int n) {
    vector<int> arr(n);
    srand(12345);
    for (int i = 0; i < n; ++i)
        arr[i] = rand() % 1000000;
    return arr;
}


void cpuBitonicSort(vector<int>& arr, int n) {
    for (int k = 2; k <= n; k *= 2) {
        for (int j = k/2; j > 0; j /= 2) {
            for (int i = 0; i < n; ++i) {
                int ixj = i ^ j;
                if (ixj > i) {
                    if ((i & k) == 0) {
                        if (arr[i] > arr[ixj]) swap(arr[i], arr[ixj]);
                    } else {
                        if (arr[i] < arr[ixj]) swap(arr[i], arr[ixj]);
                    }
                }
            }
        }
    }
}


int main() {

    vector<int> sizes = {1<<10, 1<<15, 1<<20}; // 1024, 32768, 1048576
    int runs = 10;

    cout << "CUDA Bitonic Sort Benchmark\n\n";

    for (int n : sizes) {
        cout << "Array size: " << n << "\n";

        vector<int> h_arr = generateRandomArray(n);
        int* d_arr;
        cudaMalloc(&d_arr, n * sizeof(int));
        cudaMemcpy(d_arr, h_arr.data(), n * sizeof(int), cudaMemcpyHostToDevice);

        int threadsPerBlock = 512;
        int blocks = (n + threadsPerBlock - 1) / threadsPerBlock;

        float totalTime = 0.0f;

        for (int r = 0; r < runs; ++r) {

            cudaMemcpy(d_arr, h_arr.data(), n * sizeof(int), cudaMemcpyHostToDevice);


            cudaEvent_t start, stop;
            cudaEventCreate(&start);
            cudaEventCreate(&stop);
            cudaEventRecord(start);


            for (int k = 2; k <= n; k *= 2) {
                for (int j = k/2; j > 0; j /= 2) {
                    bitonicKernel<<<blocks, threadsPerBlock>>>(d_arr, j, k, n);
                    cudaDeviceSynchronize();
                }
            }

            cudaEventRecord(stop);
            cudaEventSynchronize(stop);
            float milliseconds = 0;
            cudaEventElapsedTime(&milliseconds, start, stop);
            totalTime += milliseconds;

            cout << "  Run " << (r+1) << ": " << milliseconds << " ms\n";

            cudaEventDestroy(start);
            cudaEventDestroy(stop);
        }

        cout << "  Average time: " << (totalTime / runs) << " ms\n\n";


        cudaFree(d_arr);
    }


    vector<int> demo(8);
    cout << "Demo: Enter 8 integers:\n";
    for (int i = 0; i < 8; ++i)
        cin >> demo[i];

    cpuBitonicSort(demo, demo.size());

    cout << "Sorted array:\n";
    for (int i = 0; i < demo.size(); ++i) cout << demo[i] << " ";
    cout << "\n";

    return 0;
}


Overwriting cuda_bitonic_final.cu


In [ ]:
!nvcc -o cuda_bitonic_final cuda_bitonic_final.cu


In [ ]:
!./cuda_bitonic_final


CUDA Bitonic Sort Benchmark

Array size: 1024
  Run 1: 52.1925 ms
  Run 2: 0.060608 ms
  Run 3: 0.060736 ms
  Run 4: 0.077632 ms
  Run 5: 0.060928 ms
  Run 6: 0.060192 ms
  Run 7: 0.065344 ms
  Run 8: 0.063136 ms
  Run 9: 0.06192 ms
  Run 10: 0.0608 ms
  Average time: 5.27638 ms

Array size: 32768
  Run 1: 0.1448 ms
  Run 2: 0.126816 ms
  Run 3: 0.129024 ms
  Run 4: 0.127808 ms
  Run 5: 0.198752 ms
  Run 6: 0.162144 ms
  Run 7: 0.139936 ms
  Run 8: 0.146848 ms
  Run 9: 0.119488 ms
  Run 10: 0.098528 ms
  Average time: 0.139414 ms

Array size: 1048576
  Run 1: 0.1568 ms
  Run 2: 0.1648 ms
  Run 3: 0.169216 ms
  Run 4: 0.168032 ms
  Run 5: 0.156864 ms
  Run 6: 0.15648 ms
  Run 7: 0.155936 ms
  Run 8: 0.15584 ms
  Run 9: 0.156192 ms
  Run 10: 0.15584 ms
  Average time: 0.1596 ms

Demo: Enter 8 integers:
^C
